# 17 - HBase Fundamentals

Apache HBase - rozproszona, kolumnowa baza danych NoSQL zbudowana na HDFS.

**Tematy:**
- Architektura HBase: RegionServer, ZooKeeper, WAL, MemStore, HFile
- HBase Shell - podstawowe operacje CRUD
- Model danych: tabela, row key, column family, column qualifier, timestamp
- Projektowanie row key (hotspotting, salting, hashing)
- Spark + HBase - connector i operacje
- Zapis i odczyt danych MovieLens
- Benchmark: HBase vs PostgreSQL vs Spark JDBC

## 1. Architektura HBase

HBase to rozproszona baza danych typu **wide-column store**, zaprojektowana do przechowywania miliardów wierszy i milionów kolumn. Dziala na szczycie HDFS.

```
                          ┌─────────────┐
                          │   HMaster   │  ← koordynacja, DDL, load balancing
                          └──────┬──────┘
                                 │
              ┌──────────────────┼──────────────────┐
              │                  │                   │
     ┌────────┴────────┐ ┌──────┴────────┐ ┌───────┴───────┐
     │  RegionServer 1 │ │ RegionServer 2│ │ RegionServer 3│
     │                 │ │               │ │               │
     │ ┌─────────────┐ │ │ ┌───────────┐ │ │ ┌───────────┐ │
     │ │  Region A   │ │ │ │ Region C  │ │ │ │ Region E  │ │
     │ │ (rows a-m)  │ │ │ │(rows a-k) │ │ │ │(rows a-z) │ │
     │ ├─────────────┤ │ │ ├───────────┤ │ │ └───────────┘ │
     │ │  Region B   │ │ │ │ Region D  │ │ │               │
     │ │ (rows n-z)  │ │ │ │(rows l-z) │ │ │               │
     │ └─────────────┘ │ │ └───────────┘ │ │               │
     └─────────────────┘ └───────────────┘ └───────────────┘
              │                  │                   │
     ┌────────┴──────────────────┴───────────────────┴──────┐
     │                      ZooKeeper                       │
     │  ← leader election, RegionServer tracking, config    │
     └──────────────────────────────────────────────────────┘
              │                  │                   │
     ┌────────┴──────────────────┴───────────────────┴──────┐
     │                        HDFS                          │
     │              (trwale przechowywanie danych)          │
     └──────────────────────────────────────────────────────┘
```

### Kluczowe komponenty:

| Komponent | Rola |
|-----------|------|
| **HMaster** | Koordynacja klastra, DDL (create/alter/drop table), balansowanie regionów |
| **RegionServer** | Obsługuje odczyt/zapis dla przypisanych regionów |
| **Region** | Ciągły zakres wierszy tabeli (automatyczny split przy wzroście) |
| **ZooKeeper** | Leader election, śledzenie żywych RegionServerów, przechowywanie metadanych |
| **HDFS** | Trwale przechowuje dane (HFiles) i logi (WAL) |

### Ścieżka zapisu w HBase (Write Path)

```
  Client PUT
      │
      ▼
  ┌──────────────────────────────────────────┐
  │            RegionServer                   │
  │                                           │
  │  1. ──► WAL (Write-Ahead Log)             │  ← zapis sekwencyjny na HDFS
  │         (trwałość - odtworzenie po crash)  │     (szybki - append only)
  │                                           │
  │  2. ──► MemStore                          │  ← bufor w pamięci RAM
  │         (sortowane dane w pamięci)         │     (sorted by row key)
  │                                           │
  │  3. ──► HFile (flush gdy MemStore pełny)  │  ← trwały plik na HDFS
  │         (SSTable format, sorted, indexed)  │     (immutable!)
  │                                           │
  │  4. ──► Compaction                        │  ← łączenie małych HFiles
  │         (Minor: merge kilku HFiles)        │     w większe (optymalizacja)
  │         (Major: merge WSZYSTKICH HFiles)   │
  └──────────────────────────────────────────┘
```

### Ścieżka odczytu (Read Path)

```
  Client GET
      │
      ▼
  1. BlockCache (RAM)  ──► HIT? → zwróć dane
      │ MISS
      ▼
  2. MemStore (RAM)    ──► jest? → zwróć dane
      │ brak
      ▼
  3. HFile (HDFS)      ──► Bloom Filter → Block Index → odczytaj blok
```

**Wniosek:** HBase jest zoptymalizowany pod **szybki zapis** (WAL + MemStore) i **szybki odczyt po kluczu** (BlockCache + Bloom Filter).

## 2. Model danych HBase

HBase ma unikalny, wielowymiarowy model danych:

```
Table: movies
┌──────────┬────────────────────────────────┬────────────────────────────┐
│ Row Key  │   Column Family: info           │   Column Family: stats     │
│          ├──────────┬──────────┬──────────┤├──────────┬────────────────┤
│          │ info:title│info:year │info:genres││stats:avg │stats:count     │
├──────────┼──────────┼──────────┼──────────┼┼──────────┼────────────────┤
│ movie_1  │Toy Story │ 1995     │Animation ││ 3.92     │ 49695          │
│          │ @t=100   │ @t=100   │ @t=100   ││ @t=200   │ @t=200         │
├──────────┼──────────┼──────────┼──────────┼┼──────────┼────────────────┤
│ movie_2  │Jumanji   │ 1995     │Adventure ││ 3.21     │ 22243          │
│          │ @t=100   │ @t=100   │ @t=100   ││ @t=200   │ @t=200         │
└──────────┴──────────┴──────────┴──────────┴┴──────────┴────────────────┘
```

### Hierarchia:
```
Table
 └── Row Key (unikalny identyfikator wiersza)
      └── Column Family (deklarowana przy tworzeniu tabeli)
           └── Column Qualifier (dynamiczny - dowolna liczba)
                └── Timestamp (wersjonowanie - domyślnie 3 wersje)
                     └── Value (bajty)
```

### Kluczowe cechy:
- **Row Key** - jedyny indeks! Dane sortowane leksykograficznie po row key
- **Column Family** - deklarowana z góry, każda CF ma osobne HFile
- **Column Qualifier** - dynamiczny, nie trzeba deklarować (schema-on-read)
- **Timestamp** - automatyczne wersjonowanie, przechowuje N ostatnich wersji
- **Wszystko to bajty** - HBase nie ma typów danych (interpretacja po stronie klienta)

## 3. HBase Shell - podstawowe operacje

HBase Shell to interaktywna konsola Ruby do zarządzania HBase.

Poniższe komendy uruchamiamy w kontenerze HBase Master.

In [ ]:
%%bash
# === DDL - zarządzanie tabelami ===

# Tworzenie tabeli z dwoma column families
# hbase shell <<EOF
# create 'movies', 
#   {NAME => 'info', VERSIONS => 3, COMPRESSION => 'SNAPPY'},
#   {NAME => 'stats', VERSIONS => 5, TTL => 2592000}
# EOF
#
# NAME - nazwa column family
# VERSIONS - ile wersji (timestampów) przechowywać
# COMPRESSION - kompresja HFile (NONE, SNAPPY, GZ, LZO)
# TTL - Time To Live w sekundach (2592000 = 30 dni)

# Listowanie tabel
# hbase shell -n <<< "list"

# Opis tabeli
# hbase shell -n <<< "describe 'movies'"

# Wyłączenie i usunięcie tabeli
# hbase shell <<EOF
# disable 'movies'
# drop 'movies'
# EOF

# Modyfikacja tabeli (dodaj column family)
# hbase shell <<EOF
# disable 'movies'
# alter 'movies', {NAME => 'tags', VERSIONS => 1}
# enable 'movies'
# EOF

echo "HBase Shell DDL commands reference (uncomment to run)"

In [ ]:
%%bash
# === DML - operacje na danych ===

# PUT - wstaw/aktualizuj wartość
# hbase shell <<EOF
# put 'movies', 'movie_0001', 'info:title', 'Toy Story'
# put 'movies', 'movie_0001', 'info:year', '1995'
# put 'movies', 'movie_0001', 'info:genres', 'Animation|Children|Comedy'
# put 'movies', 'movie_0001', 'stats:avg_rating', '3.92'
# put 'movies', 'movie_0001', 'stats:num_ratings', '49695'
#
# put 'movies', 'movie_0002', 'info:title', 'Jumanji'
# put 'movies', 'movie_0002', 'info:year', '1995'
# put 'movies', 'movie_0002', 'info:genres', 'Adventure|Children|Fantasy'
# put 'movies', 'movie_0002', 'stats:avg_rating', '3.21'
# put 'movies', 'movie_0002', 'stats:num_ratings', '22243'
# EOF

# GET - odczyt jednego wiersza
# hbase shell -n <<< "get 'movies', 'movie_0001'"

# GET z filtrem column family
# hbase shell -n <<< "get 'movies', 'movie_0001', {COLUMN => 'info'}"

# GET konkretnej kolumny
# hbase shell -n <<< "get 'movies', 'movie_0001', {COLUMN => 'info:title'}"

# GET z wersjami
# hbase shell -n <<< "get 'movies', 'movie_0001', {COLUMN => 'stats:avg_rating', VERSIONS => 3}"

echo "HBase Shell DML commands reference (uncomment to run)"

In [ ]:
%%bash
# === SCAN - przeszukiwanie zakresu wierszy ===

# Scan całej tabeli (uwaga na dużych tabelach!)
# hbase shell -n <<< "scan 'movies', {LIMIT => 5}"

# Scan z zakresem row key (STARTROW inclusive, STOPROW exclusive)
# hbase shell -n <<< "scan 'movies', {STARTROW => 'movie_0001', STOPROW => 'movie_0010'}"

# Scan z filtrem kolumn
# hbase shell -n <<< "scan 'movies', {COLUMNS => ['info:title', 'stats:avg_rating'], LIMIT => 10}"

# Scan z filtrem wartości (ValueFilter)
# hbase shell <<EOF
# scan 'movies', {FILTER => "ValueFilter(=, 'substring:Animation')"}
# EOF

# Scan z PrefixFilter (filtrowanie po prefixie row key)
# hbase shell <<EOF
# scan 'movies', {FILTER => "PrefixFilter('movie_000')"}
# EOF

# DELETE - usuwanie
# hbase shell -n <<< "delete 'movies', 'movie_0002', 'stats:avg_rating'"
# hbase shell -n <<< "deleteall 'movies', 'movie_0002'"

# COUNT
# hbase shell -n <<< "count 'movies'"

echo "HBase Shell SCAN/DELETE commands reference (uncomment to run)"

## 4. Projektowanie Row Key

Row key to **najważniejsza decyzja** w projektowaniu schematu HBase. Dane są fizycznie sortowane po row key i dystrybuowane do regionów na tej podstawie.

### Problem: Hotspotting

```
Sekwencyjne row key (np. timestamp, auto-increment):

  RegionServer 1          RegionServer 2          RegionServer 3
  ┌────────────────┐      ┌────────────────┐      ┌────────────────┐
  │ Region A       │      │ Region B       │      │ Region C       │
  │ rows: 1-1000   │      │ rows: 1001-2000│      │ rows: 2001-3000│
  │ ████████████   │      │                │      │                │
  │ (HOT!)         │      │ (idle)         │      │ (idle)         │
  └────────────────┘      └────────────────┘      └────────────────┘
  
  ↑ WSZYSTKIE zapisy trafiają do jednego regionu!
```

### Rozwiązania:

| Technika | Opis | Przykład |
|----------|------|----------|
| **Salting** | Losowy prefix (0-N) | `3\|user_42\|ts` |
| **Hashing** | Hash row key | `md5(user_42)[:4]\|user_42` |
| **Reversing** | Odwrócenie klucza | `24_resu` zamiast `user_42` |
| **Composite key** | Łączenie wielu pól | `user_42\|movie_318\|ts` |

### Zasady projektowania row key:

1. **Unikaj monotonicznych kluczy** - timestamp, auto-increment = hotspot
2. **Projektuj pod najczęstsze zapytania** - row key to jedyny indeks
3. **Krótkie klucze** - każdy row key jest powtórzony w każdej komórce
4. **Salting/hashing dla zapisu** - rozprasza dane po regionach
5. **Composite key dla zapytań zakresowych** - `user_id|timestamp` umożliwia scan po user

### Przykłady row key dla MovieLens:

```
Tabela: ratings
──────────────────────────────────────────────────────────────
ZŁY:   row key = auto_increment (1, 2, 3, ...)
        → hotspot, brak możliwości query po user/movie

LEPSZY: row key = user_id|movie_id  (np. "000042|000318")
        → scan po user_id, point get po user+movie
        → ale hotspot jeśli jeden user pisze dużo

NAJLEPSZY: row key = salt|user_id|movie_id  (np. "5|000042|000318")
        → rozproszone zapisy, scan po salt+user
        → salt = hash(user_id) % num_regions

Tabela: movies
──────────────────────────────────────────────────────────────
DOBRY:  row key = movie_id (zero-padded: "000001", "000002")
        → point get po movie_id, dane stosunkowo statyczne

Tabela: user_recommendations
──────────────────────────────────────────────────────────────
DOBRY:  row key = user_id (zero-padded: "000042")
        → szybki point get rekomendacji dla użytkownika
        → CF "recs": kolumny rank_01..rank_20
```

## 5. Setup - Spark + HBase Connector

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

# Spark session z HBase connector
# shc (Spark HBase Connector) umożliwia odczyt/zapis HBase jako DataFrame
spark = SparkSession.builder \
    .appName("17_HBase_Fundamentals") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", 
            "org.postgresql:postgresql:42.7.1,"
            "org.apache.hbase.connectors.spark:hbase-spark:1.0.1,"
            "org.apache.hbase:hbase-client:2.6.1,"
            "org.apache.hbase:hbase-common:2.6.1,"
            "org.apache.hbase:hbase-mapreduce:2.6.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.hadoop.hbase.zookeeper.quorum", "hbase-zookeeper") \
    .config("spark.hadoop.hbase.zookeeper.property.clientPort", "2181") \
    .getOrCreate()

# PostgreSQL connection
jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
jdbc_props = {"user": "recommender", "password": "recommender", "driver": "org.postgresql.Driver"}

print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

In [ ]:
# Załaduj dane z PostgreSQL
ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=jdbc_props,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=jdbc_props)

ratings.cache()
movies.cache()
print(f"Ratings: {ratings.count()}, Movies: {movies.count()}")
movies.show(5, truncate=False)

## 6. Zapis danych do HBase z Spark

Spark HBase Connector (SHC) umożliwia zapis DataFrame do HBase poprzez zdefiniowanie **katalogu** - mapowania kolumn DataFrame na column families i qualifiers w HBase.

In [ ]:
import json

# Katalog SHC - definiuje mapowanie DataFrame → HBase
# Każda kolumna DataFrame jest mapowana na column family:qualifier

movies_catalog = json.dumps({
    "table": {"namespace": "default", "name": "movies"},
    "rowkey": "movie_key",
    "columns": {
        "movie_key": {"cf": "rowkey", "col": "movie_key", "type": "string"},
        "movie_id":  {"cf": "info", "col": "movie_id", "type": "int"},
        "title":     {"cf": "info", "col": "title", "type": "string"},
        "genres":    {"cf": "info", "col": "genres", "type": "string"}
    }
})

print("Movies catalog:")
print(json.dumps(json.loads(movies_catalog), indent=2))

In [ ]:
# Przygotuj dane - dodaj row key (zero-padded movie_id)
movies_for_hbase = movies.withColumn(
    "movie_key",
    lpad(col("movie_id").cast("string"), 6, "0")
)

movies_for_hbase.show(5, truncate=False)
print(f"\nPrzykładowy row key: {movies_for_hbase.first()['movie_key']}")

In [ ]:
# Zapis do HBase
# UWAGA: wymaga działającego klastra HBase!

start = time.time()

movies_for_hbase.write \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=movies_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .option("newTable", "5") \
    .mode("overwrite") \
    .save()

write_time = time.time() - start
print(f"Movies zapisane do HBase w {write_time:.1f}s")

In [ ]:
# Zapis ratings do HBase z composite row key
# Row key: salt|user_id|movie_id (rozprasza dane po regionach)

NUM_SALT_BUCKETS = 10

ratings_for_hbase = ratings.withColumn(
    "salt", (col("user_id") % NUM_SALT_BUCKETS).cast("string")
).withColumn(
    "rating_key",
    concat(
        col("salt"), lit("|"),
        lpad(col("user_id").cast("string"), 6, "0"), lit("|"),
        lpad(col("movie_id").cast("string"), 6, "0")
    )
)

ratings_for_hbase.select("rating_key", "user_id", "movie_id", "rating").show(10, truncate=False)

In [ ]:
ratings_catalog = json.dumps({
    "table": {"namespace": "default", "name": "ratings"},
    "rowkey": "rating_key",
    "columns": {
        "rating_key":       {"cf": "rowkey", "col": "rating_key", "type": "string"},
        "user_id":          {"cf": "data", "col": "user_id", "type": "int"},
        "movie_id":         {"cf": "data", "col": "movie_id", "type": "int"},
        "rating":           {"cf": "data", "col": "rating", "type": "float"},
        "rating_timestamp": {"cf": "data", "col": "rating_timestamp", "type": "string"}
    }
})

start = time.time()

ratings_for_hbase.select(
    "rating_key", "user_id", "movie_id", "rating",
    col("rating_timestamp").cast("string")
).write \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=ratings_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .option("newTable", "5") \
    .mode("overwrite") \
    .save()

write_time = time.time() - start
print(f"Ratings zapisane do HBase w {write_time:.1f}s")

## 7. Odczyt danych z HBase do Spark DataFrame

In [ ]:
# Odczyt movies z HBase
movies_from_hbase = spark.read \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=movies_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .load()

print(f"Movies z HBase: {movies_from_hbase.count()} rows")
movies_from_hbase.show(10, truncate=False)

In [ ]:
# Odczyt ratings z HBase
ratings_from_hbase = spark.read \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=ratings_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .load()

print(f"Ratings z HBase: {ratings_from_hbase.count()} rows")
ratings_from_hbase.show(5, truncate=False)

In [ ]:
# Filtrowanie po row key w HBase (predicate pushdown)
# Scan tylko wierszy z row key zaczynającym się od konkretnego salt + user

# Rekomendacje dla user_id=42 (salt = 42 % 10 = 2)
user_42_ratings = ratings_from_hbase.filter(
    col("rating_key").startswith("2|000042|")
)

print("Oceny użytkownika 42 z HBase:")
user_42_ratings.select("user_id", "movie_id", "rating") \
    .join(movies, "movie_id") \
    .select("title", "rating") \
    .orderBy(desc("rating")) \
    .show(10, truncate=False)

## 8. Benchmark: HBase GET vs PostgreSQL SELECT vs Spark JDBC

Porównajmy wydajność trzech podejść do odczytu danych:
- **HBase GET** - odczyt po row key (point lookup)
- **PostgreSQL SELECT** - zapytanie SQL po kluczu głównym
- **Spark JDBC** - odczyt przez Spark z PostgreSQL

In [ ]:
import time

def benchmark(name, func, runs=5):
    """Uruchom funkcję wielokrotnie i zmierz średni czas."""
    times = []
    for i in range(runs):
        start = time.time()
        result = func()
        elapsed = time.time() - start
        times.append(elapsed)
    avg_time = sum(times) / len(times)
    min_time = min(times)
    print(f"{name:<40} avg={avg_time*1000:.1f}ms  min={min_time*1000:.1f}ms  (runs={runs})")
    return avg_time

print("=" * 80)
print("BENCHMARK: Point lookup - pobranie jednego wiersza")
print("=" * 80)

In [ ]:
# 1. HBase GET - odczyt jednego filmu po row key
def hbase_get_movie():
    return spark.read \
        .format("org.apache.hadoop.hbase.spark") \
        .options(catalog=movies_catalog) \
        .option("hbase.spark.use.hbasecontext", "false") \
        .load() \
        .filter(col("movie_key") == "000001") \
        .collect()

hbase_time = benchmark("HBase GET (movie_key=000001)", hbase_get_movie)

# 2. PostgreSQL SELECT - zapytanie SQL
def pg_select_movie():
    return spark.read.jdbc(
        jdbc_url,
        "(SELECT * FROM movielens.movies WHERE movie_id = 1) AS t",
        properties=jdbc_props
    ).collect()

pg_time = benchmark("PostgreSQL SELECT (movie_id=1)", pg_select_movie)

# 3. Spark z cache
movies.cache()
movies.count()  # force cache

def spark_cached_get():
    return movies.filter(col("movie_id") == 1).collect()

spark_time = benchmark("Spark cached filter (movie_id=1)", spark_cached_get)

In [ ]:
print("\n" + "=" * 80)
print("BENCHMARK: Range scan - pobranie wszystkich ocen użytkownika")
print("=" * 80)

# HBase scan z prefixem row key
def hbase_scan_user():
    return spark.read \
        .format("org.apache.hadoop.hbase.spark") \
        .options(catalog=ratings_catalog) \
        .option("hbase.spark.use.hbasecontext", "false") \
        .load() \
        .filter(col("rating_key").startswith("2|000042|")) \
        .count()

hbase_scan_time = benchmark("HBase SCAN (user_42 ratings)", hbase_scan_user)

# PostgreSQL
def pg_scan_user():
    return spark.read.jdbc(
        jdbc_url,
        "(SELECT * FROM movielens.ratings WHERE user_id = 42) AS t",
        properties=jdbc_props
    ).count()

pg_scan_time = benchmark("PostgreSQL SELECT (user_id=42)", pg_scan_user)

# Spark cached
ratings.cache()
ratings.count()

def spark_scan_user():
    return ratings.filter(col("user_id") == 42).count()

spark_scan_time = benchmark("Spark cached filter (user_id=42)", spark_scan_user)

In [ ]:
print("\n" + "=" * 80)
print("BENCHMARK: Full table scan + aggregation")
print("=" * 80)

# HBase full scan
def hbase_full_scan():
    return spark.read \
        .format("org.apache.hadoop.hbase.spark") \
        .options(catalog=ratings_catalog) \
        .option("hbase.spark.use.hbasecontext", "false") \
        .load() \
        .groupBy("movie_id") \
        .agg(avg("rating"), count("*")) \
        .count()

hbase_full_time = benchmark("HBase full scan + groupBy", hbase_full_scan, runs=3)

# PostgreSQL
def pg_full_scan():
    return spark.read.jdbc(
        jdbc_url, "movielens.ratings", properties=jdbc_props,
        column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
    ).groupBy("movie_id").agg(avg("rating"), count("*")).count()

pg_full_time = benchmark("PostgreSQL JDBC full scan + groupBy", pg_full_scan, runs=3)

# Spark cached
def spark_full_scan():
    return ratings.groupBy("movie_id").agg(avg("rating"), count("*")).count()

spark_full_time = benchmark("Spark cached full scan + groupBy", spark_full_scan, runs=3)

### Podsumowanie benchmarku

| Operacja | HBase | PostgreSQL | Spark (cached) | Najlepszy |
|----------|-------|------------|----------------|----------|
| **Point lookup** (1 wiersz) | Bardzo szybki | Szybki | Szybki | HBase |
| **Range scan** (user ratings) | Szybki (jeśli dobry row key) | Szybki (z indeksem) | Szybki | Porównywalny |
| **Full scan + aggregation** | Wolny (nie do tego stworzony!) | Umiarkowany | Najszybszy | Spark |

**Wniosek:** HBase najlepiej sprawdza się do **point lookups** i **range scans po row key**. Do analityki i agregatów lepszy jest Spark z Parquet/HDFS lub SQL z PostgreSQL.

## Zadanie 1

Zaprojektuj schemat HBase dla kompletnego systemu rekomendacji MovieLens.

Wymagania:
1. Tabela `movies` - informacje o filmach (tytuł, rok, gatunki, średnia ocena)
2. Tabela `user_profiles` - profil użytkownika (ulubione gatunki, liczba ocen, średnia ocena)
3. Tabela `recommendations` - top 20 rekomendacji per użytkownik
4. Tabela `ratings` - oceny użytkowników (musi obsługiwać szybki scan po user_id)

Dla każdej tabeli zdefiniuj:
- Row key (uzasadnij wybór)
- Column families i qualifiers
- Parametry: VERSIONS, TTL, COMPRESSION
- Przewidywane access patterns (najczęstsze zapytania)

In [ ]:
# Twoje rozwiązanie:
# Zdefiniuj katalogi SHC dla każdej tabeli i uzasadnij decyzje projektowe


## Zadanie końcowe

Załaduj wszystkie dane MovieLens do HBase z odpowiednim projektem row key.

1. Utwórz tabelę `ml_movies` z CF `info` i `stats`:
   - Row key: zero-padded movie_id (np. `000001`)
   - `info:title`, `info:genres`
   - `stats:avg_rating`, `stats:num_ratings` (policz z tabeli ratings)

2. Utwórz tabelę `ml_ratings` z CF `data`:
   - Row key: `salt|user_id|movie_id` z salt = user_id % 10
   - `data:rating`, `data:timestamp`

3. Utwórz tabelę `ml_user_profiles` z CF `profile` i `activity`:
   - Row key: zero-padded user_id (np. `000042`)
   - `profile:num_ratings`, `profile:avg_rating`, `profile:top_genre`
   - `activity:first_rating_date`, `activity:last_rating_date`

4. Zweryfikuj dane:
   - Odczytaj z HBase i porównaj z PostgreSQL (count, sample rows)
   - Zmierz czas zapisu i odczytu

5. Wykonaj przykładowe zapytania:
   - Pobierz profil użytkownika 42
   - Pobierz informacje o filmie Toy Story
   - Pobierz wszystkie oceny użytkownika 42

In [ ]:
# Twoje rozwiązanie:


In [ ]:
ratings.unpersist()
movies.unpersist()
spark.stop()